In [134]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import operator
import nltk
import sklearn
import h5py
from sklearn.model_selection import train_test_split
from IPython.display import display
import random 

In [26]:
data_raw = pd.read_csv('train.csv')

In [112]:
identity_columns = ['asian', 'atheist', 'bisexual', 'black', 'buddhist', 'christian', 'female', 
                    'heterosexual', 'hindu',
                    'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
                    'jewish', 'latino', 'male', 'muslim', 'transgender', 'white']


identity_columns2 = ['_asian', '_atheist', '_bisexual', '_black', '_buddhist', '_christian', '_female', 
                    '_heterosexual', '_hindu',
                    '_homosexual_gay_or_lesbian', '_intellectual_or_learning_disability',
                    '_jewish', '_latino', '_male', '_muslim', '_transgender', '_white']
keep = ['comment_text','asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'transgender', 'white']

In [113]:
data_raw = data_raw.dropna()
y = data_raw.target
X = data_raw[keep]

In [114]:
X['comment_text'] = X['comment_text'].astype(str)

for col in identity_columns:
    X[col] = np.where(X[col] >= 0.5, True, False)

y = np.where(y >= 0.5, True, False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [115]:
X.reset_index(drop = True, inplace = True)
tempy = pd.DataFrame(y.copy())
tempy.columns = ['toxicity']

In [116]:
toxicity_rate_per_identity = np.zeros(len(identity_columns))
for i in range(len(identity_columns)):
    c = identity_columns[i]
    temp1 = X[X[c] == True]
    n = len(temp1)
    
    temp2 = tempy.loc[list(temp1.index)]
    k = len(temp2[temp2['toxicity'] == True])
    toxicity_rate_per_identity[i] = k/n

In [117]:
toxicity_rate_per_identity

array([0.11666667, 0.10851927, 0.21176471, 0.30497164, 0.09972299,
       0.08101616, 0.12909184, 0.21841542, 0.1043956 , 0.27434354,
       0.09615385, 0.14920319, 0.18704453, 0.14298477, 0.21215296,
       0.19750367, 0.27636831])

In [118]:
X_train, X_test_real, y_train, y_test = train_test_split(X, y, test_size=0.20)
X_test = X_test_real['comment_text']


In [232]:
len(toxicity_rate_per_identity)

17

In [231]:
X_train_final.shape

(188069, 1014)

In [119]:
# edited from preprocessSentences.py script from HW1
import nltk, re, pprint
from nltk import word_tokenize
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, isdir, join
import numpy
import re
import sys
import getopt
import codecs
import time
import os
import csv

chars = ['{','}','#','%','&','\(','\)','\[','\]','<','>',',', '!', '.', ';', 
'?', '*', '\\', '\/', '~', '_','|','=','+','^',':','\"','\'','@','-']

def tokenize_corpus(essay, train=True):

  porter = nltk.PorterStemmer() # also lancaster stemmer
  wnl = nltk.WordNetLemmatizer()
  stopWords = stopwords.words("english")
  classes = []
  samples = []
  docs = []
  if train == True:
    words = {}

  lines = essay.tolist()

  for line in lines:
    raw = line
    raw = ' '.join(raw.rsplit()[1:-1])
    # remove noisy characters; tokenize
    raw = re.sub(r'\n', ' ', raw)
    raw = re.sub("<", "(", raw)
    raw = re.sub(">", ")", raw)
    raw = re.sub("[\(\[].*?[\)\]]", "", raw)
    raw = re.sub('[%s]' % ''.join(chars), '', raw)
    #print(raw)
    #raw = re.sub('\n', '', raw)
    tokens = word_tokenize(raw)
    tokens = [w.lower() for w in tokens]
    tokens = [w for w in tokens if w not in stopWords]
    tokens = [wnl.lemmatize(t) for t in tokens]
    tokens = [porter.stem(t) for t in tokens]   
    if train == True:
     for t in tokens: 
         try:
             words[t] = words[t]+1
         except:
             words[t] = 1
    docs.append(tokens)

  if train == True:
     return(docs, classes, samples, words)
  else:
     return(docs, classes, samples)


def wordcount_filter(words, num=1):
   keepset = []
   for k in words.keys():
       if(words[k] > num):
           keepset.append(k)
   print ("Vocab length:", len(keepset))
   return(sorted(set(keepset)))


def find_wordcounts(docs, vocab):
   bagofwords = numpy.zeros(shape=(len(docs),len(vocab)), dtype=numpy.uint8)
   vocabIndex={}
   for i in range(len(vocab)):
      vocabIndex[vocab[i]]=i

   for i in range(len(docs)):
       doc = docs[i]

       for t in doc:
          index_t=vocabIndex.get(t)
          if index_t != None:
             bagofwords[i,index_t]=bagofwords[i,index_t]+1

   print ("Finished find_wordcounts for:", len(docs), "docs")
   return(bagofwords)

In [120]:
(docs, classes, samples, words) = tokenize_corpus(X_train.comment_text, train=True)

In [121]:
sorted_x = sorted(words.items(), key=operator.itemgetter(1))
word_count_threshold = sorted_x[-1000][1]
vocab = wordcount_filter(words, num=word_count_threshold)

print('word and word count threshold is:')
print(sorted_x[-1000])

Vocab length: 997
word and word count threshold is:
('threaten', 1101)


In [122]:
bow1 = find_wordcounts(docs, vocab)
df1 = pd.DataFrame(bow1, columns = vocab)


Finished find_wordcounts for: 188069 docs


In [123]:
#####for noidentitybias analysis #####
# we have X train called 'df1' and y train called y_train
# we now need to create X_test using the vocabulary selected from X_train, aka the columns of df1
# fit df1 using y_train, use the fitted model to predict y_test from X_test, then compare with real values of y_test


#we have X_test, where each row is a comment. 
# we generate new list of words in each comment in X_test
# we find the wordcounts of the new "docs" using the vocab list generated by X_train

(docs2, classes2, samples2, words2) = tokenize_corpus(X_test, train=True)
bow2 = find_wordcounts(docs2, vocab)
df2 = pd.DataFrame(bow2, columns = vocab)

Finished find_wordcounts for: 47018 docs


In [177]:



X_test_notext = X_test_real.copy().drop(['comment_text'], axis=1)
s = X_test_notext.columns.to_series()
X_test_notext.columns = ('_')+s
X_test_notext.reset_index(drop=True, inplace=True)
X_test_final = pd.concat([X_test_notext, df2], axis=1)


X_train_notext = X_train.copy().drop(['comment_text'], axis=1)
s = X_train_notext.columns.to_series()
X_train_notext.columns = ('_')+s
X_train_notext.reset_index(drop=True, inplace=True)
X_train_final = pd.concat([X_train_notext, df1], axis=1)



In [125]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


clf_log_noidentity = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train_final.copy().drop(identity_columns2, axis = 1), y_train)
    
y_predict_log_noidentity = clf_log_noidentity.predict(X_test_final.copy().drop(identity_columns2, axis = 1))

log_score_noidentity = roc_auc_score(y_test, y_predict_log_noidentity)

print(log_score_noidentity)
print(clf_log_noidentity.score(X_train_final.copy().drop(identity_columns2, axis=1), y_train))
print(clf_log_noidentity.score(X_test_final.copy().drop(identity_columns2, axis = 1), y_test))


0.7159682347926457
0.7720942845445022
0.7706410310944745


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score


clf_rfc_noidentity = RandomForestClassifier(n_estimators = 50, max_depth = 3).fit(X_train_final.copy().drop(identity_columns2, axis = 1), y_train)
    
y_predict_rfc_noidentity = clf_rfc_noidentity.predict(X_test_final.copy().drop(identity_columns2, axis = 1))

rfc_score_noidentity = roc_auc_score(y_test, y_predict_rfc_noidentity)

print(rfc_score_noidentity)
print(clf_rfc_noidentity.score(X_train_final.copy().drop(identity_columns2, axis=1), y_train))
print(clf_rfc_noidentity.score(X_test_final.copy().drop(identity_columns2, axis = 1), y_test))

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score


clf_svc_noidentity = LinearSVC().fit(X_train_final.copy().drop(identity_columns2, axis = 1), y_train)
    
y_predict_svc_noidentity = clf_svc_noidentity.predict(X_test_final.copy().drop(identity_columns2, axis = 1))

svc_score_noidentity = roc_auc_score(y_test, y_predict_svc_noidentity)

print(svc_score_noidentity)
print(clf_svc_noidentity.score(X_train_final.copy().drop(identity_columns2, axis=1), y_train))
print(clf_svc_noidentity.score(X_test_final.copy().drop(identity_columns2, axis = 1), y_test))

In [178]:
###########compute overall AUC #######
print('Overall AUC is:')
print(log_score_noidentity)

##########LOG#
###########compute overall AUC #######
print('Overall AUC is:')
print(log_score_noidentity)

############compute Subgroup AUC #####
subgroupAUC_log = np.zeros(len(identity_columns))

for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    #print(c)
    temp = X_test_final[X_test_final[c] == True]
    #print(temp.shape)
    if len(temp) == 0:
        subgroupAUC_log[i] = 1
    else:
        X_test_final_sub1 = temp.drop(identity_columns2, axis = 1)
        y_predict_sub1 = clf_log_noidentity.predict(X_test_final_sub1)
        y_test_sub1 = y_test[list(X_test_final_sub1.index)]
        log_score_sub1 = roc_auc_score(y_test_sub1, y_predict_sub1)
        subgroupAUC_log[i] = log_score_sub1

            
    
subgroupAUC_log_mean = np.mean(subgroupAUC_log)
print('Subgroup AUC is:')
print(subgroupAUC_log_mean)
###compute Subgroup AUC #####
subgroupAUC_log = np.zeros(len(identity_columns))

for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    #print(c)
    temp = X_test_final[X_test_final[c] == True]
    #print(temp.shape)
    if len(temp) == 0:
        subgroupAUC_log[i] = 1
    else:
        X_test_final_sub1 = temp.drop(identity_columns2, axis = 1)
        y_predict_sub1 = clf_log_noidentity.predict(X_test_final_sub1)
        y_test_sub1 = y_test[list(X_test_final_sub1.index)]
        log_score_sub1 = roc_auc_score(y_test_sub1, y_predict_sub1)
        subgroupAUC_log[i] = log_score_sub1

            
    
subgroupAUC_log_mean = np.mean(subgroupAUC_log)
print('Subgroup AUC is:')
print(subgroupAUC_log_mean)


Overall AUC is:
0.7159682347926457
Subgroup AUC is:
0.6953131508730317


In [ ]:
#RFC#
############compute Subgroup AUC #####
subgroupAUC_rfc = np.zeros(len(identity_columns))

for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    #print(c)
    temp = X_test_final[X_test_final[c] == True]
    #print(temp.shape)
    if len(temp) == 0:
        subgroupAUC_rfc[i] = 1
    else:
        X_test_final_sub1 = temp.drop(identity_columns2, axis = 1)
        y_predict_sub1 = clf_rfc_noidentity.predict(X_test_final_sub1)
        y_test_sub1 = y_test[list(X_test_final_sub1.index)]
        rfc_score_sub1 = roc_auc_score(y_test_sub1, y_predict_sub1)
        subgroupAUC_rfc[i] = rfc_score_sub1

            
    
subgroupAUC_rfc_mean = np.mean(subgroupAUC_rfc)
print('Subgroup AUC is:')
print(subgroupAUC_rfc_mean)


In [ ]:
#SVC#
############compute Subgroup AUC #####
subgroupAUC_svc = np.zeros(len(identity_columns))

for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    #print(c)
    temp = X_test_final[X_test_final[c] == True]
    #print(temp.shape)
    if len(temp) == 0:
        subgroupAUC_svc[i] = 1
    else:
        X_test_final_sub1 = temp.drop(identity_columns2, axis = 1)
        y_predict_sub1 = clf_svc_noidentity.predict(X_test_final_sub1)
        y_test_sub1 = y_test[list(X_test_final_sub1.index)]
        svc_score_sub1 = roc_auc_score(y_test_sub1, y_predict_sub1)
        subgroupAUC_svc[i] = svc_score_sub1

            
    
subgroupAUC_svc_mean = np.mean(subgroupAUC_svc)
print('Subgroup AUC is:')
print(subgroupAUC_svc_mean)


In [ ]:
#LOG#
############compute BPSN Background Positive, Subgroup Negative AUC#####
BPSN_log = np.zeros(len(identity_columns))
y_test1 = pd.DataFrame(y_test)
y_test1.columns = ['toxicity']
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    temp1 = y_test1[y_test1['toxicity'] == False]
    temp2 = X_test_final.loc[list(temp1.index)]
    temp_pt1 = temp2[temp2[c] == True]
    
    temp3 = y_test1[y_test1['toxicity'] == True]
    temp4 = X_test_final.loc[list(temp3.index)]
    temp_pt2 = temp4[temp4[c] == False]
    
    X_test_final_sub2 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    X_test_final_sub2 = X_test_final_sub2.drop(identity_columns2, axis =1)
    
    y_predict_sub2 = clf_log_noidentity.predict(X_test_final_sub2)
    y_test_sub2 = y_test[list(X_test_final_sub2.index)]
    
    log_score_sub2 = roc_auc_score(y_test_sub2, y_predict_sub2)
    BPSN_log[i] = log_score_sub2
    
BPSN_log_mean = np.mean(BPSN_log)
print('BPSN is:')
print(BPSN_log_mean)
    

In [ ]:
#RFC#
############compute BPSN Background Positive, Subgroup Negative AUC#####
BPSN_rfc = np.zeros(len(identity_columns))
y_test1 = pd.DataFrame(y_test)
y_test1.columns = ['toxicity']
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    temp1 = y_test1[y_test1['toxicity'] == False]
    temp2 = X_test_final.loc[list(temp1.index)]
    temp_pt1 = temp2[temp2[c] == True]
    
    temp3 = y_test1[y_test1['toxicity'] == True]
    temp4 = X_test_final.loc[list(temp3.index)]
    temp_pt2 = temp4[temp4[c] == False]
    
    X_test_final_sub2 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    X_test_final_sub2 = X_test_final_sub2.drop(identity_columns2, axis =1)
    
    y_predict_sub2 = clf_rfc_noidentity.predict(X_test_final_sub2)
    y_test_sub2 = y_test[list(X_test_final_sub2.index)]
    
    rfc_score_sub2 = roc_auc_score(y_test_sub2, y_predict_sub2)
    BPSN_rfc[i] = rfc_score_sub2
    
BPSN_rfc_mean = np.mean(BPSN_rfc)
print('BPSN is:')
print(BPSN_rfc_mean)
    

In [ ]:
#SVC#
############compute BPSN Background Positive, Subgroup Negative AUC#####
BPSN_svc = np.zeros(len(identity_columns))
y_test1 = pd.DataFrame(y_test)
y_test1.columns = ['toxicity']
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    temp1 = y_test1[y_test1['toxicity'] == False]
    temp2 = X_test_final.loc[list(temp1.index)]
    temp_pt1 = temp2[temp2[c] == True]
    
    temp3 = y_test1[y_test1['toxicity'] == True]
    temp4 = X_test_final.loc[list(temp3.index)]
    temp_pt2 = temp4[temp4[c] == False]
    
    X_test_final_sub2 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    X_test_final_sub2 = X_test_final_sub2.drop(identity_columns2, axis =1)
    
    y_predict_sub2 = clf_svc_noidentity.predict(X_test_final_sub2)
    y_test_sub2 = y_test[list(X_test_final_sub2.index)]
    
    svc_score_sub2 = roc_auc_score(y_test_sub2, y_predict_sub2)
    BPSN_svc[i] = svc_score_sub2
    
BPSN_svc_mean = np.mean(BPSN_svc)
print('BPSN is:')
print(BPSN_svc_mean)
    

In [ ]:
#LOG#
############compute BNSP Background Negative, Subgroup Positive AUC#####
BNSP_log = np.zeros(len(identity_columns))
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    temp1 = y_test1[y_test1['toxicity'] == True]
    temp2 = X_test_final.loc[list(temp1.index)]
    temp_pt1 = temp2[temp2[c] == True]
    
    temp3 = y_test1[y_test1['toxicity'] == False]
    temp4 = X_test_final.loc[list(temp3.index)]
    temp_pt2 = temp4[temp4[c] == False]
    
    X_test_final_sub2 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    X_test_final_sub2 = X_test_final_sub2.drop(identity_columns2, axis = 1)
    
    y_predict_sub2 = clf_log_noidentity.predict(X_test_final_sub2)
    y_test_sub2 = y_test[list(X_test_final_sub2.index)]
    
    log_score_sub2 = roc_auc_score(y_test_sub2, y_predict_sub2)
    BNSP_log[i] = log_score_sub2
    
BNSP_log_mean = np.mean(BNSP_log)
print('BNSP is:')
print(BNSP_log_mean)


In [ ]:
#RFC#
############compute BNSP Background Negative, Subgroup Positive AUC#####
BNSP_rfc = np.zeros(len(identity_columns))
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    temp1 = y_test1[y_test1['toxicity'] == True]
    temp2 = X_test_final.loc[list(temp1.index)]
    temp_pt1 = temp2[temp2[c] == True]
    
    temp3 = y_test1[y_test1['toxicity'] == False]
    temp4 = X_test_final.loc[list(temp3.index)]
    temp_pt2 = temp4[temp4[c] == False]
    
    X_test_final_sub2 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    X_test_final_sub2 = X_test_final_sub2.drop(identity_columns2, axis = 1)
    
    y_predict_sub2 = clf_rfc_noidentity.predict(X_test_final_sub2)
    y_test_sub2 = y_test[list(X_test_final_sub2.index)]
    
    rfc_score_sub2 = roc_auc_score(y_test_sub2, y_predict_sub2)
    BNSP_rfc[i] = rfc_score_sub2
    
BNSP_rfc_mean = np.mean(BNSP_rfc)
print('BNSP is:')
print(BNSP_rfc_mean)


In [ ]:
#SVC#
############compute BNSP Background Negative, Subgroup Positive AUC#####
BNSP_svc = np.zeros(len(identity_columns))
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    temp1 = y_test1[y_test1['toxicity'] == True]
    temp2 = X_test_final.loc[list(temp1.index)]
    temp_pt1 = temp2[temp2[c] == True]
    
    temp3 = y_test1[y_test1['toxicity'] == False]
    temp4 = X_test_final.loc[list(temp3.index)]
    temp_pt2 = temp4[temp4[c] == False]
    
    X_test_final_sub2 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    X_test_final_sub2 = X_test_final_sub2.drop(identity_columns2, axis = 1)
    
    y_predict_sub2 = clf_svc_noidentity.predict(X_test_final_sub2)
    y_test_sub2 = y_test[list(X_test_final_sub2.index)]
    
    svc_score_sub2 = roc_auc_score(y_test_sub2, y_predict_sub2)
    BNSP_svc[i] = svc_score_sub2
    
BNSP_svc_mean = np.mean(BNSP_svc)
print('BNSP is:')
print(BNSP_svc_mean)


In [ ]:
#LOG#
############Final Metric ####################
score_log = 0.25*log_score_noidentity + 0.25*subgroupAUC_log_mean + 0.25*BPSN_log_mean + 0.25*BNSP_log_mean
print('Final Metric is:')
print(score_log)


print('with p = -5')
print('overall auc is')
subgroupAUC_log_inverse = np.divide(1.0, subgroupAUC_log)
part1 = (1/np.mean(np.power(subgroupAUC_log_inverse, 5))) ** 0.2
print('subgroup auc is')
print(part1)
BPSN_log_inverse = np.divide(1.0, BPSN_log)
part2 = (1/np.mean(np.power(BPSN_log_inverse, 5))) ** 0.2
print('bpsn is')
print(part2)
BNSP_log_inverse = np.divide(1.0, BNSP_log)
part3 = (1/np.mean(np.power(BNSP_log_inverse, 5))) ** 0.2
print('bnsp is')
print(part3)
print('final metric is')
print(0.25*log_score_noidentity + 0.25*part1 + 0.25*part2 + 0.25*part3)


In [ ]:
#RFC#
############Final Metric ####################
score_rfc = 0.25*rfc_score_noidentity + 0.25*subgroupAUC_rfc_mean + 0.25*BPSN_rfc_mean + 0.25*BNSP_rfc_mean
print('Final Metric is:')
print(score_rfc)


print('with p = -5')
print('overall auc is')
subgroupAUC_rfc_inverse = np.divide(1.0, subgroupAUC_rfc)
part1 = (1/np.mean(np.power(subgroupAUC_rfc_inverse, 5))) ** 0.2
print('subgroup auc is')
print(part1)
BPSN_rfc_inverse = np.divide(1.0, BPSN_rfc)
part2 = (1/np.mean(np.power(BPSN_rfc_inverse, 5))) ** 0.2
print('bpsn is')
print(part2)
BNSP_rfc_inverse = np.divide(1.0, BNSP_rfc)
part3 = (1/np.mean(np.power(BNSP_rfc_inverse, 5))) ** 0.2
print('bnsp is')
print(part3)
print('final metric is')
print(0.25*rfc_score_noidentity + 0.25*part1 + 0.25*part2 + 0.25*part3)


In [ ]:
#SVC#
############Final Metric ####################
score_svc = 0.25*svc_score_noidentity + 0.25*subgroupAUC_svc_mean + 0.25*BPSN_svc_mean + 0.25*BNSP_svc_mean
print('Final Metric is:')
print(score_svc)


print('with p = -5')
print('overall auc is')
subgroupAUC_svc_inverse = np.divide(1.0, subgroupAUC_svc)
part1 = (1/np.mean(np.power(subgroupAUC_svc_inverse, 5))) ** 0.2
print('subgroup auc is')
print(part1)
BPSN_svc_inverse = np.divide(1.0, BPSN_svc)
part2 = (1/np.mean(np.power(BPSN_svc_inverse, 5))) ** 0.2
print('bpsn is')
print(part2)
BNSP_svc_inverse = np.divide(1.0, BNSP_svc)
part3 = (1/np.mean(np.power(BNSP_svc_inverse, 5))) ** 0.2
print('bnsp is')
print(part3)
print('final metric is')
print(0.25*svc_score_noidentity + 0.25*part1 + 0.25*part2 + 0.25*part3)


In [153]:
X_test_final.head()

,_asian,_atheist,_bisexual,_black,_buddhist,_christian,_female,_heterosexual,_hindu,_homosexual_gay_or_lesbian,...,wrote,ye,year,yet,young,your,youv,’,“,”
0,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
1,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
2,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,1,0,0,0
3,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
4,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,1,0,0,0,0


In [181]:
# X_train_final is the full dataset with both bag of words and identity representation
# identity_columns2 is a vector of column names that represent identities

from sklearn.linear_model import LogisticRegression


predicted_identities = np.zeros((len(X_test_final),len(identity_columns2)))

for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    print(c)
    temp_true = X_train_final[X_train_final[c] == True]
    n = len(temp_true)
    
    temp_false = X_train_final[X_train_final[c] == False]
    temp_false_sampled = temp_false.loc[random.sample(list(temp_false.index), n)]
    
    temp_final = pd.concat([temp_true, temp_false_sampled], axis=0)
    
    # the true identity in X_train we are trying to predict
    temp_y = temp_final[c].astype('bool')
    # the bag of words corresponding to the selected balanced dataset
    temp_X = temp_final.copy().drop(identity_columns2, axis = 1)
    
    
    ##################### Log Predict ##################################################
    # fit logistic regression to predict the identity 'c' from the selected balanced dataset
    clf_log = LogisticRegression(random_state=0, solver='lbfgs').fit(temp_X, temp_y)
    
    # predict identity 'c' of the Test dataset, df2 represents the bag of words representation of X_test
    predicted_identity = clf_log.predict(X_test_final.copy().drop(identity_columns2, axis = 1))
    predicted_identities[:,i] = predicted_identity.astype('bool')
    
 
    

predicted_identities= pd.DataFrame(predicted_identities, columns = identity_columns2)
s = X_train_notext.columns.to_series()
predicted_identities.columns = s
predicted_identities.reset_index(drop = True, inplace = True)

X_test_estimated = pd.concat([predicted_identities, df2], axis=1)


_asian
_atheist
_bisexual
_black
_buddhist
_christian
_female
_heterosexual
_hindu
_homosexual_gay_or_lesbian
_intellectual_or_learning_disability
_jewish
_latino
_male
_muslim
_transgender
_white


In [193]:
X_test_estimated.head()

,_asian,_atheist,_bisexual,_black,_buddhist,_christian,_female,_heterosexual,_hindu,_homosexual_gay_or_lesbian,...,wrote,ye,year,yet,young,your,youv,’,“,”
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0


In [184]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


clf_log_identity = LogisticRegression(random_state=0, solver='lbfgs', class_weight = 'balanced').fit(X_train_final, y_train)
    
y_predict_log_identity = clf_log_identity.predict(X_test_estimated)

log_score_identity = roc_auc_score(y_test, y_predict_log_identity)

print(log_score_identity)
print(clf_log_identity.score(X_train_final, y_train))
print(clf_log_identity.score(X_test_estimated, y_test))


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.6934326288926116
0.7818088042154742
0.6589816665957718


In [191]:
###########compute overall AUC #######
print('Overall AUC is:')
print(log_score_identity)

############compute Subgroup AUC #####
subgroupAUC_log = np.zeros(len(identity_columns))

for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    #print(c)
    temp1 = X_test_final[X_test_final[c] == True]
    temp = X_test_estimated.loc[list(temp1.index)]
    #print(temp.shape)
    if len(temp) == 0:
        subgroupAUC_log[i] = 1
    else:
        y_predict_sub1 = clf_log_identity.predict(temp)
        y_test_sub1 = y_test[list(temp.index)]
        log_score_sub1 = roc_auc_score(y_test_sub1, y_predict_sub1)
        subgroupAUC_log[i] = log_score_sub1

            
    
subgroupAUC_log_mean = np.mean(subgroupAUC_log)
print('Subgroup AUC is:')
print(subgroupAUC_log_mean)


Overall AUC is:
0.6934326288926116
Subgroup AUC is:
0.6560870717184137


In [216]:
############compute BPSN Background Positive, Subgroup Negative AUC#####
BPSN_log = np.zeros(len(identity_columns))
y_test1 = pd.DataFrame(y_test)
y_test1.columns = ['toxicity']
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    temp_pt1 = X_test_estimated.loc[(y_test1['toxicity'] == False) & (X_test_final[c] == True)]
    temp_pt2 = X_test_estimated.loc[(y_test1['toxicity'] == True) & (X_test_final[c] == False)]
    
    
    X_test_final_sub2 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    
    y_predict_sub2 = clf_log_identity.predict(X_test_final_sub2)
    y_test_sub2 = y_test[list(X_test_final_sub2.index)]
    
    log_score_sub2 = roc_auc_score(y_test_sub2, y_predict_sub2)
    BPSN_log[i] = log_score_sub2
    
BPSN_log_mean = np.mean(BPSN_log)
print('BPSN is:')
print(BPSN_log_mean)
    

BPSN is:
0.5904212832834496


In [219]:
############compute BNSP Background Negative, Subgroup Positive AUC#####
BNSP_log = np.zeros(len(identity_columns))
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    
    temp_pt1 = X_test_estimated.loc[(y_test1['toxicity'] == True) & (X_test_final[c] == True)]
    temp_pt2 = X_test_estimated.loc[(y_test1['toxicity'] == False) & (X_test_final[c] == False)]

    
    X_test_final_sub3 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    
    y_predict_sub3 = clf_log_identity.predict(X_test_final_sub3)
    y_test_sub3 = y_test[list(X_test_final_sub3.index)]
    
    log_score_sub3 = roc_auc_score(y_test_sub3, y_predict_sub3)
    BNSP_log[i] = log_score_sub3
    
BNSP_log_mean = np.mean(BNSP_log)
print('BNSP is:')
print(BNSP_log_mean)


BNSP is:
0.7579372243836852


In [222]:
############Final Metric ####################
score_log = 0.25*log_score_noidentity + 0.25*subgroupAUC_log_mean + 0.25*BPSN_log_mean + 0.25*BNSP_log_mean
print('Final Metric is:')
print(score_log)


print('with p = -5')
print('overall auc is')
subgroupAUC_log_inverse = np.divide(1.0, subgroupAUC_log)
part1 = (1/np.mean(np.power(subgroupAUC_log_inverse, 5))) ** 0.2
print('subgroup auc is')
print(part1)
BPSN_log_inverse = np.divide(1.0, BPSN_log)
part2 = (1/np.mean(np.power(BPSN_log_inverse, 5))) ** 0.2
print('bpsn is')
print(part2)
BNSP_log_inverse = np.divide(1.0, BNSP_log)
part3 = (1/np.mean(np.power(BNSP_log_inverse, 5))) ** 0.2
print('bnsp is')
print(part3)
print('final metric is')
print(0.25*log_score_identity + 0.25*part1 + 0.25*part2 + 0.25*part3)


Final Metric is:
0.6801034535445485
with p = -5
overall auc is
subgroup auc is
0.6323676755902574
bpsn is
0.5419928470351245
bnsp is
0.7486216616619104
final metric is
0.6541037032949759


In [223]:
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score

clf_SVC_identity = LinearSVC(class_weight = 'balanced').fit(X_train_final, y_train)
y_predict_SVC_identity = clf_SVC_identity.predict(X_test_estimated)
SVC_score_identity = roc_auc_score(y_test, y_predict_log_identity)

print(SVC_score_identity)
print(clf_SVC_identity.score(X_train_final, y_train))
print(clf_SVC_identity.score(X_test_estimated, y_test))

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6934326288926116
0.793097214320276
0.6645540005955166


In [224]:
###########compute overall AUC #######
print('Overall AUC is:')
print(SVC_score_identity)

############compute Subgroup AUC #####
subgroupAUC_SVC = np.zeros(len(identity_columns))

for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    #print(c)
    temp1 = X_test_final[X_test_final[c] == True]
    temp = X_test_estimated.loc[list(temp1.index)]
    #print(temp.shape)
    if len(temp) == 0:
        subgroupAUC_SVC[i] = 1
    else:
        y_predict_sub1 = clf_SVC_identity.predict(temp)
        y_test_sub1 = y_test[list(temp.index)]
        SVC_score_sub1 = roc_auc_score(y_test_sub1, y_predict_sub1)
        subgroupAUC_SVC[i] = SVC_score_sub1

            
    
subgroupAUC_SVC_mean = np.mean(subgroupAUC_SVC)
print('Subgroup AUC is:')
print(subgroupAUC_SVC_mean)

############compute BPSN Background Positive, Subgroup Negative AUC#####
BPSN_SVC = np.zeros(len(identity_columns))
y_test1 = pd.DataFrame(y_test)
y_test1.columns = ['toxicity']
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    temp_pt1 = X_test_estimated.loc[(y_test1['toxicity'] == False) & (X_test_final[c] == True)]
    temp_pt2 = X_test_estimated.loc[(y_test1['toxicity'] == True) & (X_test_final[c] == False)]
    
    
    X_test_final_sub2 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    
    y_predict_sub2 = clf_SVC_identity.predict(X_test_final_sub2)
    y_test_sub2 = y_test[list(X_test_final_sub2.index)]
    
    SVC_score_sub2 = roc_auc_score(y_test_sub2, y_predict_sub2)
    BPSN_SVC[i] = SVC_score_sub2
    
BPSN_SVC_mean = np.mean(BPSN_SVC)
print('BPSN is:')
print(BPSN_SVC_mean)
    
############compute BNSP Background Negative, Subgroup Positive AUC#####
BNSP_SVC = np.zeros(len(identity_columns))
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    
    temp_pt1 = X_test_estimated.loc[(y_test1['toxicity'] == True) & (X_test_final[c] == True)]
    temp_pt2 = X_test_estimated.loc[(y_test1['toxicity'] == False) & (X_test_final[c] == False)]

    
    X_test_final_sub3 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    
    y_predict_sub3 = clf_SVC_identity.predict(X_test_final_sub3)
    y_test_sub3 = y_test[list(X_test_final_sub3.index)]
    
    SVC_score_sub3 = roc_auc_score(y_test_sub3, y_predict_sub3)
    BNSP_SVC[i] = SVC_score_sub3
    
BNSP_SVC_mean = np.mean(BNSP_SVC)
print('BNSP is:')
print(BNSP_SVC_mean)

############Final Metric ####################
score_SVC = 0.25*SVC_score_identity + 0.25*subgroupAUC_SVC_mean + 0.25*BPSN_SVC_mean + 0.25*BNSP_SVC_mean
print('Final Metric is:')
print(score_SVC)


print('with p = -5')
print('overall auc is')
subgroupAUC_SVC_inverse = np.divide(1.0, subgroupAUC_SVC)
part1 = (1/np.mean(np.power(subgroupAUC_SVC_inverse, 5))) ** 0.2
print('subgroup auc is')
print(part1)
BPSN_SVC_inverse = np.divide(1.0, BPSN_SVC)
part2 = (1/np.mean(np.power(BPSN_SVC_inverse, 5))) ** 0.2
print('bpsn is')
print(part2)
BNSP_SVC_inverse = np.divide(1.0, BNSP_SVC)
part3 = (1/np.mean(np.power(BNSP_SVC_inverse, 5))) ** 0.2
print('bnsp is')
print(part3)
print('final metric is')
print(0.25*SVC_score_identity + 0.25*part1 + 0.25*part2 + 0.25*part3)


Overall AUC is:
0.6934326288926116
Subgroup AUC is:
0.6527879103240851
BPSN is:
0.5851957837893274
BNSP is:
0.7592490261010133
Final Metric is:
0.6726663372767594
with p = -5
overall auc is
subgroup auc is
0.6302810484996727
bpsn is
0.5367802597077723
bnsp is
0.7498143008937661
final metric is
0.6525770594984557


In [225]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

clf_RF_identity = RandomForestClassifier(n_estimators=50, random_state=0, class_weight = 'balanced')
clf_RF_identity.fit(X_train_final, y_train) 

y_predict_RF_identity = clf_RF_identity.predict(X_test_estimated)
RF_score_identity = roc_auc_score(y_test, y_predict_RF_identity)

print(RF_score_identity)
print(clf_RF_identity.score(X_train_final, y_train))
print(clf_RF_identity.score(X_test_estimated, y_test))

0.5554915014766049
0.9908703720443028
0.8849589518907652


In [226]:
###########compute overall AUC #######
print('Overall AUC is:')
print(RF_score_identity)

############compute Subgroup AUC #####
subgroupAUC_RF = np.zeros(len(identity_columns))

for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    #print(c)
    temp1 = X_test_final[X_test_final[c] == True]
    temp = X_test_estimated.loc[list(temp1.index)]
    #print(temp.shape)
    if len(temp) == 0:
        subgroupAUC_RF[i] = 1
    else:
        y_predict_sub1 = clf_RF_identity.predict(temp)
        y_test_sub1 = y_test[list(temp.index)]
        RF_score_sub1 = roc_auc_score(y_test_sub1, y_predict_sub1)
        subgroupAUC_RF[i] = RF_score_sub1

            
    
subgroupAUC_RF_mean = np.mean(subgroupAUC_RF)
print('Subgroup AUC is:')
print(subgroupAUC_RF_mean)

############compute BPSN Background Positive, Subgroup Negative AUC#####
BPSN_RF = np.zeros(len(identity_columns))
y_test1 = pd.DataFrame(y_test)
y_test1.columns = ['toxicity']
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    temp_pt1 = X_test_estimated.loc[(y_test1['toxicity'] == False) & (X_test_final[c] == True)]
    temp_pt2 = X_test_estimated.loc[(y_test1['toxicity'] == True) & (X_test_final[c] == False)]
    
    
    X_test_final_sub2 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    
    y_predict_sub2 = clf_RF_identity.predict(X_test_final_sub2)
    y_test_sub2 = y_test[list(X_test_final_sub2.index)]
    
    RF_score_sub2 = roc_auc_score(y_test_sub2, y_predict_sub2)
    BPSN_RF[i] = RF_score_sub2
    
BPSN_RF_mean = np.mean(BPSN_RF)
print('BPSN is:')
print(BPSN_RF_mean)
    
############compute BNSP Background Negative, Subgroup Positive AUC#####
BNSP_RF = np.zeros(len(identity_columns))
for i in range(len(identity_columns2)):
    c = identity_columns2[i]
    
    temp_pt1 = X_test_estimated.loc[(y_test1['toxicity'] == True) & (X_test_final[c] == True)]
    temp_pt2 = X_test_estimated.loc[(y_test1['toxicity'] == False) & (X_test_final[c] == False)]

    
    X_test_final_sub3 = pd.concat([temp_pt1, temp_pt2], axis = 0)
    
    y_predict_sub3 = clf_RF_identity.predict(X_test_final_sub3)
    y_test_sub3 = y_test[list(X_test_final_sub3.index)]
    
    RF_score_sub3 = roc_auc_score(y_test_sub3, y_predict_sub3)
    BNSP_RF[i] = RF_score_sub3
    
BNSP_RF_mean = np.mean(BNSP_RF)
print('BNSP is:')
print(BNSP_RF_mean)

############Final Metric ####################
score_RF = 0.25*RF_score_identity + 0.25*subgroupAUC_RF_mean + 0.25*BPSN_RF_mean + 0.25*BNSP_RF_mean
print('Final Metric is:')
print(score_RF)


print('with p = -5')
print('overall auc is')
subgroupAUC_RF_inverse = np.divide(1.0, subgroupAUC_RF)
part1 = (1/np.mean(np.power(subgroupAUC_RF_inverse, 5))) ** 0.2
print('subgroup auc is')
print(part1)
BPSN_RF_inverse = np.divide(1.0, BPSN_RF)
part2 = (1/np.mean(np.power(BPSN_RF_inverse, 5))) ** 0.2
print('bpsn is')
print(part2)
BNSP_RF_inverse = np.divide(1.0, BNSP_RF)
part3 = (1/np.mean(np.power(BNSP_RF_inverse, 5))) ** 0.2
print('bnsp is')
print(part3)
print('final metric is')
print(0.25*RF_score_identity + 0.25*part1 + 0.25*part2 + 0.25*part3)


Overall AUC is:
0.5554915014766049
Subgroup AUC is:
0.5310262855649387
BPSN is:
0.5571816150595875
BNSP is:
0.5299833447118536
Final Metric is:
0.5434206867032463
with p = -5
overall auc is
subgroup auc is
0.5274653726319837
bpsn is
0.556334094987835
bnsp is
0.5270890078649004
final metric is
0.541594994240331
